In [502]:
import json 
import os
import logging 
import concurrent 
import asyncio 
import re 
import requests 

from collections import deque 
from itertools import chain 
from datetime import date, datetime 

from typing import List 
from pprint import PrettyPrinter
from subgrounds import Subgrounds
from subgrounds.pagination import ShallowStrategy
from IPython.display import HTML, display
from web3 import Web3
from functools import partial 
from concurrent.futures import ThreadPoolExecutor
from gql import gql, Client
from gql.transport.aiohttp import AIOHTTPTransport

import pandas as pd 
import numpy as np 
import altair as alt 
import missingno as miss

# logging.basicConfig(level=logging.INFO)

# TODO: Figure out how this frax subgraph is useful
# https://api.thegraph.com/subgraphs/name/frax-finance-data/fraxbp-subgraph/graphql

pp = PrettyPrinter().pprint

In [427]:
URL_INFURA = 'https://mainnet.infura.io/v3/856c3834f317452a82e25bb06e04de18'
w3 = Web3(Web3.HTTPProvider(URL_INFURA))

### Bribes Analysis 

We are interested in analyzing the impact of bribes through 

- Votium (voting power vlCVX, incentives in FXS) 
- Pitch 
- bribe.crv.finance 
- Hidden Hand 
- Aura 

that incentivize liquidity in liquidity pools that contain FRAX. To start, we are working with the following subset of pools that exist on curve. 
- Metapools (stable / non-stable). 
- Frax BP
- Frax:3crv

The following metrics are of interest

- \\$ Bribes spent per pool (broken down by platform and incentive type)  
- \\$ Liquidity per pool (broken into Frax and non-Frax components) 

Once we have both the liquidity and bribe information, we can try to answer these questions
- Where are bribe dollars best spent? 
- How does the Frax protocol optimize its bribing strategy in order to maximize the amount of liquidity within it's pools? 

In [504]:
def ddf(df):
    display(HTML(df.to_html()))
    
    
def remove_prefix(df: pd.DataFrame, prefix: str):
    # Remove a prefix from all columns 
    cols = [c for c in df.columns]
    for i, c in enumerate(cols): 
        if c.startswith(prefix): 
            cols[i] = c[len(prefix):]
    df.columns = cols 
    return df 

def remove_prefixes(df: pd.DataFrame, prefixes: List[str]):
    for p in prefixes: 
        df = remove_prefix(df, p)
    return df 

def zip_dfs(dfs, col_names): 
    data = dfs[0]
    for i, c in enumerate(col_names): 
        data = data.merge(dfs[i+1][[c]], how='left', left_index=True, right_index=True)
    return data

def recursive_index_merge(dfs):
    drop_right_suffix = '__drop_right_suffix'
    assert all(len(dfs[0]) == len(dfs[i]) for i in range(1, len(dfs)))
    dfs = deque(dfs) 
    df = dfs.popleft()
    while dfs: 
        df_right = dfs.popleft()
        cols = list(set(df.columns).intersection(df_right.columns))
        df = df.merge(df_right, how='left', left_index=True, right_index=True, suffixes=(None, drop_right_suffix))
        drop_cols = [col for col in df.columns if col.endswith(drop_right_suffix)]
        df = df.drop(columns=drop_cols)
    return df 

def query_attrs(query, attrs):
    qattrs = []
    for a in attrs: 
        if '.' in a: 
            # nested
            v = None 
            for p in a.split('.'): 
                v = getattr(query, p) if v is None else getattr(v, p) 
            qattrs.append(v) 
        else: 
            # non-nested 
            qattrs.append(getattr(query, a)) 
    return sg.query_df(qattrs, pagination_strategy=ShallowStrategy) 

def camel_to_snake(name):
    name = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', name).lower()

def df_cols_camel_to_snake(df):
    df.columns = [camel_to_snake(c) for c in df.columns]
    return df 

def df_cols_change_prefix(df, prefix_cur, prefix_new):
    df.columns = [
        prefix_new + c[len(prefix_cur):] if c.startswith(prefix_cur) else c for c in df.columns 
    ]
    return df 

async def graphql_execute(
    query, 
    variable_values=None, 
    paginate=False, 
    page_size=10000, 
    page_size_variable='page_size', 
    page_offset_variable='page_offset',
    verbose=False, 
):
    variable_values = variable_values or {}
    transport = AIOHTTPTransport(url=URL_SNAPSHOT)
    # Using `async with` on the client will start a connection on the transport
    # and provide a `session` variable to execute queries on this connection
    async with Client(transport=transport, fetch_schema_from_transport=True) as session:
        if not paginate:
            gquery = gql(query)
            result = await session.execute(gquery)
            if verbose: 
                logging.info(f"Query returned {len(result)} records with page size {page_size}.")
            return result 
        else: 
            results = []
            finished = False 
            i = 0 
            while not finished:
                gquery = gql(query)
                # TODO: Could be a failure point but works for single queries 
                query_name = gquery.to_dict()['definitions'][0]['name']['value'].lower()
                query_resp = await session.execute(
                    gquery, variable_values={
                        **variable_values, page_offset_variable: page_size * i, page_size_variable: page_size
                    }
                )
                result = query_resp[query_name]
                if verbose: 
                    logging.info(f"Paginated query returned {len(result)} records with page size {page_size}.")
                finished = len(result) == 0
                results.append(result)
                i += 1
            return list(chain(*results)) 

In [505]:
# Non-subgraph
URL_SNAPSHOT = 'https://hub.snapshot.org/graphql'
# Subgraph 
URL_VOTIUM = 'https://api.thegraph.com/subgraphs/name/convex-community/votium-bribes' 
URL_CURVE_POOLS = 'https://api.thegraph.com/subgraphs/name/convex-community/curve-pools'
URL_CURVE_VOL_MAINNET = 'https://api.thegraph.com/subgraphs/name/convex-community/volume-mainnet' 

# Addresses 
VOTIUM_VOTER = '0xde1E6A7ED0ad3F61D531a8a78E83CcDdbd6E0c49'.lower()
CURVE_POOL_FRAX_USDC = '0xdcef968d416a41cdac0ed8702fac8128a64241a2'
ADDRESS_FXS = '0x3432B6A60D23Ca0dFCa7761B7ab56459D9C964D0'.lower()
ADDRESS_CRVFRAX = '0x3175df0976dfa876431c2e9ee6bc45b65d3473cc'.lower()
ADDRESS_FRAXBP_POOL = '0xdcef968d416a41cdac0ed8702fac8128a64241a2'.lower()

In [430]:
# Instantiate the client with an endpoint.
sg = Subgrounds()

# votium_bribes = sg.load_subgraph(URL_VOTIUM) 
sg_curve_pools = sg.load_subgraph(URL_CURVE_POOLS) 
sg_curve_vol = sg.load_subgraph(URL_CURVE_VOL_MAINNET)
sg_votium = sg.load_subgraph(URL_VOTIUM) 

### Metapool TVL  

In [431]:
# Discovery of all curve metapools that have gauges 
# We will later get all pools paired with the fraxBP, and join in this extra gauge data for the relevant pools. 
query_metapools = sg_curve_pools.Query.pools(first=100, where={'coins_contains': [ADDRESS_CRVFRAX]})
attrs = [
    'id',
    'name',
    'gauge',
    'token', # convex receipt token. TODO: How does this work? 
    'lpToken', 
    'swap', # pool contract address 
    'coins',
    'assetType', 
]
df_metapools_gauges = query_attrs(query_metapools, attrs)
df_metapools_gauges['pool_coin_index'] = df_metapools_gauges.groupby(['pools_id']).cumcount() + 1
df_metapools_gauges.pools_assetType = df_metapools_gauges.pools_assetType.apply(lambda v: {0: "STABLE", 4: "CRYPTO"}[int(v)])
df_metapools_gauges = df_cols_camel_to_snake(df_metapools_gauges) 
df_metapools_gauges = df_cols_change_prefix(df_metapools_gauges, "pools", "pool") 
assert len(df_metapools_gauges.pool_id.unique()) == len(df_metapools_gauges.pool_swap.unique()), "Detected duplicate metapools with gauges for FraxBP" 
df_metapools_gauges = df_metapools_gauges.rename(columns={
    'pool_token': 'pool_cvx_token',
    'pool_swap': 'pool_address',
    'pool_coins': 'pool_coin_address',
    'pool_asset_type': 'pool_type', 
}).drop(columns=['pool_id'])
df_metapools_gauges.head()

,pool_name,pool_gauge,pool_cvx_token,pool_lp_token,pool_address,pool_coin_address,pool_type,pool_coin_index
0,Curve.fi Factory USD Metapool: sUSDFRAXBP,0xf6d7087d4ae4dcf85956d743406e63cda74d99ad,0x8e2a6e9390cbd4c3895d07e4cb171c0527990df6,0xe3c190c57b5959ae62efe3b6797058b76ba2f5ef,0xe3c190c57b5959ae62efe3b6797058b76ba2f5ef,0x57ab1ec28d129707052df4df418d58a2d46d5f51,STABLE,1
1,Curve.fi Factory USD Metapool: sUSDFRAXBP,0xf6d7087d4ae4dcf85956d743406e63cda74d99ad,0x8e2a6e9390cbd4c3895d07e4cb171c0527990df6,0xe3c190c57b5959ae62efe3b6797058b76ba2f5ef,0xe3c190c57b5959ae62efe3b6797058b76ba2f5ef,0x3175df0976dfa876431c2e9ee6bc45b65d3473cc,STABLE,2
2,Curve.fi Factory USD Metapool: LUSDFRAXBP,0x389fc079a15354e9cbce8258433cc0f85b755a42,0xe8a371b5d32344033589a2f0a2712dbd12130b18,0x497ce58f34605b9944e6b15ecafe6b001206fd25,0x497ce58f34605b9944e6b15ecafe6b001206fd25,0x5f98805a4e8be255a32880fdec7f6728c6568ba0,STABLE,1
3,Curve.fi Factory USD Metapool: LUSDFRAXBP,0x389fc079a15354e9cbce8258433cc0f85b755a42,0xe8a371b5d32344033589a2f0a2712dbd12130b18,0x497ce58f34605b9944e6b15ecafe6b001206fd25,0x497ce58f34605b9944e6b15ecafe6b001206fd25,0x3175df0976dfa876431c2e9ee6bc45b65d3473cc,STABLE,2
4,Curve.fi Factory USD Metapool: apeUSDFRAXBP,0xd6e48cc0597a1ee12a8beeb88e22bfdb81777164,0x5ec62bad0fa0c6b7f87b3b86edfe1bcd2a3139e2,0x04b727c7e246ca70d496ecf52e6b6280f3c8077d,0x04b727c7e246ca70d496ecf52e6b6280f3c8077d,0xff709449528b6fb6b88f557f7d93dece33bca78d,STABLE,1


In [482]:
query_pools = sg_curve_vol.Query.dailyPoolSnapshots(
    first=100, where={
        'pool': ADDRESS_FRAXBP_POOL # This is the BENTFRAXBP-f that was deprecated 
    } 
)

dfs_fraxbp = query_attrs(query_pools, [
    # snapshot attributes 
    'id', 'timestamp', 'reservesUSD'
])
dfs_fraxbp.head()

,dailyPoolSnapshots_id,dailyPoolSnapshots_timestamp,dailyPoolSnapshots_reservesUSD
0,0xdcef968d416a41cdac0ed8702fac8128a64241a2-165...,1655251200,6.785477e+06
1,0xdcef968d416a41cdac0ed8702fac8128a64241a2-165...,1655251200,2.296155e+06
2,0xdcef968d416a41cdac0ed8702fac8128a64241a2-165...,1655337600,6.834332e+06
3,0xdcef968d416a41cdac0ed8702fac8128a64241a2-165...,1655337600,2.236406e+06
4,0xdcef968d416a41cdac0ed8702fac8128a64241a2-165...,1655424000,6.814277e+06


In [498]:
query_pools = sg_curve_vol.Query.dailyPoolSnapshots(
    first=100, where={
        'pool': ADDRESS_FRAXBP_POOL # This is the BENTFRAXBP-f that was deprecated 
    } 
)

dfs_fraxbp = query_attrs(query_pools, [
    # pool attributes 
    'pool.id', 'pool.name', 'pool.lpToken', 'pool.symbol', 'pool.creationBlock', 'pool.coins', 'pool.coinNames', 'pool.coinDecimals', 'pool.poolType', 
    # snapshot attributes 
    'id', 'timestamp', 'reservesUSD'
])

In [501]:
df = recursive_index_merge(dfs_fraxbp)

200


,dailyPoolSnapshots_pool_id,dailyPoolSnapshots_pool_name,dailyPoolSnapshots_pool_lpToken,dailyPoolSnapshots_pool_symbol,dailyPoolSnapshots_pool_creationBlock,dailyPoolSnapshots_pool_coins,dailyPoolSnapshots_pool_poolType,dailyPoolSnapshots_id,dailyPoolSnapshots_timestamp,dailyPoolSnapshots_pool_coinNames,dailyPoolSnapshots_pool_coinDecimals,dailyPoolSnapshots_reservesUSD
0,0xdcef968d416a41cdac0ed8702fac8128a64241a2,Curve.fi FRAX/USDC,0x3175df0976dfa876431c2e9ee6bc45b65d3473cc,crvFRAX,14967642,0x853d955acef822db058eb8505911ed77f175b99e,REGISTRY_V1,0xdcef968d416a41cdac0ed8702fac8128a64241a2-165...,1655251200,FRAX,18,6.785477e+06
1,0xdcef968d416a41cdac0ed8702fac8128a64241a2,Curve.fi FRAX/USDC,0x3175df0976dfa876431c2e9ee6bc45b65d3473cc,crvFRAX,14967642,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,REGISTRY_V1,0xdcef968d416a41cdac0ed8702fac8128a64241a2-165...,1655251200,USDC,6,2.296155e+06
2,0xdcef968d416a41cdac0ed8702fac8128a64241a2,Curve.fi FRAX/USDC,0x3175df0976dfa876431c2e9ee6bc45b65d3473cc,crvFRAX,14967642,0x853d955acef822db058eb8505911ed77f175b99e,REGISTRY_V1,0xdcef968d416a41cdac0ed8702fac8128a64241a2-165...,1655337600,FRAX,18,6.834332e+06
3,0xdcef968d416a41cdac0ed8702fac8128a64241a2,Curve.fi FRAX/USDC,0x3175df0976dfa876431c2e9ee6bc45b65d3473cc,crvFRAX,14967642,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,REGISTRY_V1,0xdcef968d416a41cdac0ed8702fac8128a64241a2-165...,1655337600,USDC,6,2.236406e+06
4,0xdcef968d416a41cdac0ed8702fac8128a64241a2,Curve.fi FRAX/USDC,0x3175df0976dfa876431c2e9ee6bc45b65d3473cc,crvFRAX,14967642,0x853d955acef822db058eb8505911ed77f175b99e,REGISTRY_V1,0xdcef968d416a41cdac0ed8702fac8128a64241a2-165...,1655424000,FRAX,18,6.814277e+06


In [472]:
# Discovery of all curve metapools (both with and without gauges) + fraxbp 

# The query above uses a different subgraph that exposes more detailed information about pools with gauges 
query_pools = sg_curve_vol.Query.pools(
    first=100, where={
        'coins_contains': [ADDRESS_CRVFRAX], 
        'cumulativeVolumeUSD_gt': 0, # There are some test pools and pools that were incorrectly created that we filter out by only getting pools with more than 0 volume 
        'id_not': '0xd3301b7caa76f932816a6fc7ef0b673238e217ad' # This is the BENTFRAXBP-f that was deprecated 
    } 
)
# query_pools = sg_curve_vol.Query.pools(
#     first=100, where={
#         'id': ADDRESS_FRAXBP_POOL # This is the BENTFRAXBP-f that was deprecated 
#     } 
# )
dfs_metapools_all = query_attrs(query_pools, [
    'id', 'name', 'lpToken', 'symbol', 'creationBlock', 'coins', 'coinNames', 'coinDecimals', 'poolType'
])
df_metapools_all = zip_dfs(dfs_metapools_all, ['pools_coinNames', 'pools_coinDecimals'])
df_metapools_all = df_metapools_all.rename(columns={'pools_id': 'pools_address'})
# There's one row in this df for each combination of a pool and a coin. So a pool with 3 coins will lead to 3 rows. 
# When subgrounds flattens data in producing output dataframes, the order of the rows is preserved so this method 
# is fine for determining the coin index (assuming we didn't do any sorting after pulling the data. 
df_metapools_all['pool_coin_index'] = df_metapools_all.groupby('pools_address').cumcount() + 1
df_metapools_all['pools_poolType'] = df_metapools_all.pools_poolType.apply(lambda v: v.replace("_FACTORY", ""))
assert set(df_metapools_all.pools_poolType.unique()) == set(['STABLE', 'CRYPTO']), "Invalid pool type detected"
assert len(df_metapools_all.pools_address.unique()) == len(df_metapools_all.pools_symbol.unique()), "Detected duplicate metapools for FraxBP" 
df_metapools_all = df_cols_camel_to_snake(df_metapools_all) 
df_metapools_all = df_cols_change_prefix(df_metapools_all, "pools", "pool") 
df_metapools_all = df_metapools_all.rename(columns={
    'pool_coins': 'pool_coin_address', 
    'pool_coin_names': 'pool_coin_name', 
    'pool_pool_type': 'pool_type'
})
df_metapools_all.head()

,pool_address,pool_name,pool_lp_token,pool_symbol,pool_creation_block,pool_coin_address,pool_type,pool_coin_name,pool_coin_decimals,pool_coin_index
0,0x02dfa5c793a9ce4d767a86259245a162a57f2db4,Curve.fi Factory Crypto Pool: bentCVX/FraxBP,0xbb23c0361d3e436fb7942a0e103edecab3afa917,bentCVXFRX-f,15531927,0x9e0441e084f5db0606565737158aa6ab6b970fe0,CRYPTO,bentCVX,18,1
1,0x02dfa5c793a9ce4d767a86259245a162a57f2db4,Curve.fi Factory Crypto Pool: bentCVX/FraxBP,0xbb23c0361d3e436fb7942a0e103edecab3afa917,bentCVXFRX-f,15531927,0x3175df0976dfa876431c2e9ee6bc45b65d3473cc,CRYPTO,crvFRAX,18,2
2,0x04b727c7e246ca70d496ecf52e6b6280f3c8077d,Curve.fi Factory USD Metapool: apeUSDFRAXBP,0x04b727c7e246ca70d496ecf52e6b6280f3c8077d,APEUSDBP3CRV-f,15049114,0xff709449528b6fb6b88f557f7d93dece33bca78d,STABLE,ApeUSD,18,1
3,0x04b727c7e246ca70d496ecf52e6b6280f3c8077d,Curve.fi Factory USD Metapool: apeUSDFRAXBP,0x04b727c7e246ca70d496ecf52e6b6280f3c8077d,APEUSDBP3CRV-f,15049114,0x3175df0976dfa876431c2e9ee6bc45b65d3473cc,STABLE,crvFRAX,18,2
4,0x13b876c26ad6d21cb87ae459eaf6d7a1b788a113,Curve.fi Factory Crypto Pool: BADGER/FRAXBP,0x09b2e090531228d1b8e3d948c73b990cb6e60720,BADGERFRAX-f,15523744,0x3472a5a71965499acd81997a54bba8d852c6e53d,CRYPTO,BADGER,18,1


In [433]:
# df_metapools_gauges.head(1)

In [434]:
# df_metapools_all.head(1)

In [435]:
def combine_pool_data(df_metapools_gauges, df_pools):
    # Join in gauge specific info to all pools in df_pools 
    merge_cols = ["pool_address", "pool_lp_token", "pool_coin_address", "pool_type", "pool_coin_index"]
    df_pools = (
        df_pools.merge(
            df_metapools_gauges[merge_cols + ['pool_gauge', 'pool_cvx_token']], 
            how='left', 
            on=merge_cols, 
            suffixes=(None, '_y')
        )
    )
    # Ensure that all pools with gauges successfully merged (no NaN's) 
    gauge_mask = df_pools.pool_address.isin(df_metapools_gauges.pool_address.unique().tolist()) 
    assert all(~df_pools.loc[gauge_mask].isna())
    # Add indicator column to mark pools that have gauges 
    df_pools['has_gauge'] = False 
    df_pools.loc[gauge_mask, 'has_gauge'] = True 
    # miss.matrix(df_pools) 
    return df_pools 
    
df_pools = combine_pool_data(df_metapools_gauges, df_metapools_all)
df_pools.head()

,pool_address,pool_name,pool_lp_token,pool_symbol,pool_creation_block,pool_coin_address,pool_type,pool_coin_name,pool_coin_decimals,pool_coin_index,pool_gauge,pool_cvx_token,has_gauge
0,0x02dfa5c793a9ce4d767a86259245a162a57f2db4,Curve.fi Factory Crypto Pool: bentCVX/FraxBP,0xbb23c0361d3e436fb7942a0e103edecab3afa917,bentCVXFRX-f,15531927,0x9e0441e084f5db0606565737158aa6ab6b970fe0,CRYPTO,bentCVX,18,1,NaN,NaN,False
1,0x02dfa5c793a9ce4d767a86259245a162a57f2db4,Curve.fi Factory Crypto Pool: bentCVX/FraxBP,0xbb23c0361d3e436fb7942a0e103edecab3afa917,bentCVXFRX-f,15531927,0x3175df0976dfa876431c2e9ee6bc45b65d3473cc,CRYPTO,crvFRAX,18,2,NaN,NaN,False
2,0x04b727c7e246ca70d496ecf52e6b6280f3c8077d,Curve.fi Factory USD Metapool: apeUSDFRAXBP,0x04b727c7e246ca70d496ecf52e6b6280f3c8077d,APEUSDBP3CRV-f,15049114,0xff709449528b6fb6b88f557f7d93dece33bca78d,STABLE,ApeUSD,18,1,0xd6e48cc0597a1ee12a8beeb88e22bfdb81777164,0x5ec62bad0fa0c6b7f87b3b86edfe1bcd2a3139e2,True
3,0x04b727c7e246ca70d496ecf52e6b6280f3c8077d,Curve.fi Factory USD Metapool: apeUSDFRAXBP,0x04b727c7e246ca70d496ecf52e6b6280f3c8077d,APEUSDBP3CRV-f,15049114,0x3175df0976dfa876431c2e9ee6bc45b65d3473cc,STABLE,crvFRAX,18,2,0xd6e48cc0597a1ee12a8beeb88e22bfdb81777164,0x5ec62bad0fa0c6b7f87b3b86edfe1bcd2a3139e2,True
4,0x13b876c26ad6d21cb87ae459eaf6d7a1b788a113,Curve.fi Factory Crypto Pool: BADGER/FRAXBP,0x09b2e090531228d1b8e3d948c73b990cb6e60720,BADGERFRAX-f,15523744,0x3472a5a71965499acd81997a54bba8d852c6e53d,CRYPTO,BADGER,18,1,0x455279344f84a496615dc0ffa0511d2e19ec19d8,0x25f0b7c3a7a43b409634a5759526560cc3313d75,True


In [436]:
"""Get daily snapshots of tvl for all of the metapools. 
"""
pool_ids = df_pools.pool_address.unique().tolist()
query_pool_snaps = sg_curve_vol.Query.dailyPoolSnapshots(first=100000, where={"pool_in": pool_ids})
df_reserves = query_attrs(query_pool_snaps, ['id', 'timestamp', 'reservesUSD'])
df_reserves.dailyPoolSnapshots_timestamp = pd.to_datetime(df_reserves.dailyPoolSnapshots_timestamp, unit='s')
df_reserves['pool_address'] = df_reserves.dailyPoolSnapshots_id.apply(lambda _id: _id.split('-')[0])
df_reserves['pool_coin_index'] = df_reserves.groupby(['dailyPoolSnapshots_id', 'dailyPoolSnapshots_timestamp']).cumcount() + 1
df_reserves = df_reserves.drop(columns=['dailyPoolSnapshots_id']) 
df_reserves = df_cols_camel_to_snake(df_reserves) 
df_reserves = df_cols_change_prefix(df_reserves, "daily_pool_snapshots_", "snapshot_")
print(len(df_reserves))
assert len(pool_ids) == len(df_reserves.pool_address.unique())

4764


In [453]:
df_reserves.head(2)

,snapshot_timestamp,snapshot_reserves_usd,pool_address,pool_coin_index
0,2022-09-14,0.0,0x02dfa5c793a9ce4d767a86259245a162a57f2db4,1
1,2022-09-14,0.0,0x02dfa5c793a9ce4d767a86259245a162a57f2db4,2


In [438]:
df_liquidity = df_reserves.merge(df_pools, how='left', on=['pool_address', 'pool_coin_index'], validate="m:1")

In [439]:
last_snapshot = df_liquidity.groupby(["pool_name", "pool_address"])['snapshot_reserves_usd'].last().reset_index()
inactive_pools = last_snapshot.loc[last_snapshot.snapshot_reserves_usd < 1]
if len(inactive_pools): 
    for p in inactive_pools.to_dict(orient="records"):
        print(f"Dropping: {p['pool_name']}")
inactive_addrs = inactive_pools.pool_address.unique()
# Create a new column indicating if a pool is active (currently has non-zero TVL). 
df_pools['pool_active'] = True 
df_pools.loc[~df_pools.pool_address.isin(inactive_addrs), 'pool_active'] = False
# Remove pools from the liquidity profile that are inactive
df_liquidity = df_liquidity.loc[~df_liquidity.pool_address.isin(inactive_addrs)]

Dropping: Curve.fi Factory Crypto Pool: RAI/FRAXBP


In [454]:
# df_liquidity.loc[df_liquidity.pool_name == 'Curve.fi Factory Crypto Pool: bentCVX/FraxBP']
df_liquidity.head(2)

,snapshot_timestamp,snapshot_reserves_usd,pool_address,pool_coin_index,pool_name,pool_lp_token,pool_symbol,pool_creation_block,pool_coin_address,pool_type,pool_coin_name,pool_coin_decimals,pool_gauge,pool_cvx_token,has_gauge
0,2022-09-14,0.0,0x02dfa5c793a9ce4d767a86259245a162a57f2db4,1,Curve.fi Factory Crypto Pool: bentCVX/FraxBP,0xbb23c0361d3e436fb7942a0e103edecab3afa917,bentCVXFRX-f,15531927,0x9e0441e084f5db0606565737158aa6ab6b970fe0,CRYPTO,bentCVX,18,NaN,NaN,False
1,2022-09-14,0.0,0x02dfa5c793a9ce4d767a86259245a162a57f2db4,2,Curve.fi Factory Crypto Pool: bentCVX/FraxBP,0xbb23c0361d3e436fb7942a0e103edecab3afa917,bentCVXFRX-f,15531927,0x3175df0976dfa876431c2e9ee6bc45b65d3473cc,CRYPTO,crvFRAX,18,NaN,NaN,False


### Metapool TVL Charts 

#### TVL Faceted by Metapool Segmented by Coin Name 

In [441]:
(
    alt.Chart(df_liquidity)
    .mark_area()
    .transform_calculate(stack_order="datum.pool_coin_name === 'crvFRAX' ? 0 : 1")
    .encode(
        x="snapshot_timestamp:T", 
        y="snapshot_reserves_usd:Q", 
        color=alt.Color("pool_coin_name:N", scale=alt.Scale(scheme="tableau20")), 
        tooltip=alt.Tooltip("snapshot_reserves_usd:Q", format='$,d'), 
        facet=alt.Facet('pool_name:N', columns=3),
        order="stack_order:O", 
    )
    .resolve_scale(y="independent")
    .resolve_axis(x="independent")
    .properties(width=250, height=150)
)
# TODO: Bent pool data is off for BENT tvl. 

/opt/miniconda3/envs/frax-bribes/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

#### Current Metapool Liquidity Faceted by Asset Type Segmented by Metapool

In [442]:
df_liquidity.head(1)

,snapshot_timestamp,snapshot_reserves_usd,pool_address,pool_coin_index,pool_name,pool_lp_token,pool_symbol,pool_creation_block,pool_coin_address,pool_type,pool_coin_name,pool_coin_decimals,pool_gauge,pool_cvx_token,has_gauge
0,2022-09-14,0.0,0x02dfa5c793a9ce4d767a86259245a162a57f2db4,1,Curve.fi Factory Crypto Pool: bentCVX/FraxBP,0xbb23c0361d3e436fb7942a0e103edecab3afa917,bentCVXFRX-f,15531927,0x9e0441e084f5db0606565737158aa6ab6b970fe0,CRYPTO,bentCVX,18,NaN,NaN,False


In [443]:
df_pools.head(1)

,pool_address,pool_name,pool_lp_token,pool_symbol,pool_creation_block,pool_coin_address,pool_type,pool_coin_name,pool_coin_decimals,pool_coin_index,pool_gauge,pool_cvx_token,has_gauge,pool_active
0,0x02dfa5c793a9ce4d767a86259245a162a57f2db4,Curve.fi Factory Crypto Pool: bentCVX/FraxBP,0xbb23c0361d3e436fb7942a0e103edecab3afa917,bentCVXFRX-f,15531927,0x9e0441e084f5db0606565737158aa6ab6b970fe0,CRYPTO,bentCVX,18,1,NaN,NaN,False,False


In [444]:
df_tvl = (
    df_liquidity[df_liquidity.pool_coin_name != 'crvFRAX']
    .groupby("pool_name", sort="snapshot_timestamp").last().reset_index()
    [['snapshot_reserves_usd', 'pool_type', 'pool_coin_name']]
)
df_tvl = df_tvl.merge(
    df_tvl.groupby('pool_type')['snapshot_reserves_usd'].sum().reset_index().rename(columns={'snapshot_reserves_usd': 'tvl_total'}), 
    how='left', on="pool_type", validate="m:1"
)
df_tvl['tvl_share'] = df_tvl.snapshot_reserves_usd / df_tvl.tvl_total
# df_tvl.head()

In [445]:
(
    alt.Chart(df_tvl)
    .mark_arc()
    .encode(
        theta="snapshot_reserves_usd:Q", 
        color=alt.Color("pool_coin_name:N", scale=alt.Scale(scheme="tableau20")), 
        facet=alt.Facet('pool_type:N', columns=3),
        tooltip=[
            alt.Tooltip("pool_coin_name:N", title="Token"), 
            alt.Tooltip("snapshot_reserves_usd:Q", format="$,d", title="TVL"),
            alt.Tooltip("tvl_share:Q", format=".1%", title="TVL Share")
        ] 
    )
    .resolve_scale(theta="independent")
)

/opt/miniconda3/envs/frax-bribes/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

#### Current Metapool Liquidity Segmented By Asset Type 

In [446]:
df_tvl_type = df_liquidity.copy()
df_tvl_type.snapshot_reserves_usd = df_tvl_type.snapshot_reserves_usd.astype(np.float64)
df_tvl_type = (
    df_tvl_type[df_tvl_type.pool_coin_name != 'crvFRAX']
    .groupby(["pool_type", "snapshot_timestamp"], dropna=False)['snapshot_reserves_usd'].sum().reset_index()
    .rename(columns={'snapshot_reserves_usd': 'tvl_type'})
)
df_tvl_total = df_tvl_type.groupby("snapshot_timestamp")['tvl_type'].sum().reset_index().rename(columns={'tvl_type': 'tvl_total'})

In [447]:
# df_tvl_type.sort_values(['snapshot_timestamp', 'pool_type']).tail(1)

In [448]:
# df_tvl_total.tail(1)

In [449]:
(
    alt.Chart(df_tvl_type)
    .mark_area()
    .encode(
        x="snapshot_timestamp:T", 
        y="tvl_type:Q", 
        color="pool_type:N", 
        tooltip=["pool_type:N", alt.Tooltip("tvl_type:Q", format="$,d")]
    ) + 
    alt.Chart(df_tvl_total)
    .mark_line(color="green")
    .encode(
        x="snapshot_timestamp:T", 
        y="tvl_total:Q", 
        tooltip=[alt.Tooltip("tvl_total:Q", format="$,d")]
    )
)

/opt/miniconda3/envs/frax-bribes/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.LayerChart(...)

### Snapshot Proposals 

Here, we get the snapshot proposals that correspond to votium votes. Each proposal contains the possible set of choices (liquidity pools that rewards can be directed to). 

In [450]:
proposal_attrs = [
    'id', 
    'title', 
    'choices', 
    'created', 
    'start', 
    'end', 
    'state', 
    'scores_total', 
    
]
proposals = await graphql_execute(
    '''
    query Proposals {
      proposals(
        first: 10000,
        where: {title_contains: "Gauge weight", space: "cvx.eth"},
        orderBy: "created",
        orderDirection: desc
      ) {
        <proposal_attrs>
      }
    }
    '''.replace('<proposal_attrs>', '\n'.join(proposal_attrs))
)

In [451]:
async def get_snapshot_proposals(): 
    """Get all snapshot proposals corresponding to convex gauge weight votes. 
    
    Within each gauge weight vote, holders of vlCVX collectively determine how convex should distribute it's 
    veCRV in subsequent curve gauge votes. 
    
    - Votes are pulled from the snapshot graphql endpoint. 
    """
    proposal_attrs = ['id', 'title', 'choices', 'start', 'end', 'state']
    proposals = await graphql_execute(
        '''
        query Proposals {
          proposals(
            first: 10000,
            where: {title_contains: "Gauge weight", space: "cvx.eth"},
            orderBy: "created",
            orderDirection: desc
          ) {
            <proposal_attrs>
          }
        }
        '''.replace('<proposal_attrs>', '\n'.join(proposal_attrs))
    )
    # Number of rounds here should match number of rounds on llama airforce 
    # https://llama.airforce/#/bribes/rounds/votium/cvx-crv/
    df_proposals = pd.DataFrame(proposals['proposals'])
    df_proposals.start = pd.to_datetime(df_proposals.start, unit='s').dt.date
    df_proposals.end = pd.to_datetime(df_proposals.end, unit='s').dt.date 
    df_proposals = df_proposals.sort_values('start').reset_index(drop=True)
    # On votium, the proposal id is keccak256 hashed. So we need to perform this operation on the data pulled from snapshot 
    df_proposals['id_keccak256'] = df_proposals.id.apply(
        lambda _id: Web3.keccak(text=_id).hex() if not _id.startswith('0x') else Web3.keccak(hexstr=_id).hex()
    )
    # Validate that this query pulls in all data by ensuring that the number of votium voting rounds we get from this query 
    # matches the expected number of voting rounds, which we can compute using simple timedelta logic, the date of the 
    # first vote, and the current date. 
    dmin = pd.Timestamp('2021-09-16') # day of the first votium snapshot 
    assert pd.Timestamp(df_proposals.start.min()) == dmin
    df_proposals['start_diff'] = df_proposals.start.shift(1) - df_proposals.start
    assert (df_proposals.iloc[1:,].start_diff == pd.Timedelta('-14 days')).all()
    # Since a proposal happens every two weeks, and we know the date of the first proposal, we can compute the number of expected proposals on any given date. 
    num_proposals_expected = (
        int(np.ceil((pd.Timestamp(pd.Timestamp.now().date()) - dmin) / pd.Timedelta("14 days")))
    )
    df_proposals['proposal_id'] = df_proposals.id 
    df_proposals = df_proposals.rename(columns={
        'title': 'proposal_title', 
        'choices': 'proposal_choices', 
        'start': 'proposal_start', 
        'end': 'proposal_end', 
        'id_keccak256': 'proposal_id_keccak256'
    })
    df_proposals = df_proposals.sort_values('proposal_start').reset_index(drop=True).reset_index()
    df_proposals['proposal_round'] = df_proposals['index'] + 1 
    df_proposals = df_proposals.drop(columns=['index', 'start_diff', 'id', 'state']) 
    assert len(df_proposals) == num_proposals_expected
    return df_proposals 
    
    
async def get_snapshot_votes(proposal_ids, verbose=True): 
    """Get all votes for all convex gauge weight snapshot proposals. 
        
    - Votes are pulled from the snapshot graphql endpoint. 
    """
    votes = await graphql_execute(
        '''
            query Votes($page_size: Int!, $page_offset: Int!, $proposal_ids: [String]!) {
              votes (
                first: $page_size
                skip: $page_offset
                where: { proposal_in: $proposal_ids }
              ) {
                proposal { id }
                id
                voter
                created
                choice
                vp
                vp_by_strategy
                vp_state
              }
            }
        ''', 
        paginate=True, 
        variable_values={'proposal_ids': proposal_ids}
    )
    vote_records = []
    for v in votes: 
        for index, amount in v['choice'].items(): 
            # TODO: data validation step / what does vp_state on this entity mean? Sometimes it's pending but still seems to be counted in votes. 
            # assert v['vp_state'] == 'final'
            r = {
                **v, 
                'choice_index': int(index) - 1, 
                'amount': amount, 
                'is_votium': v['voter'].lower() == VOTIUM_VOTER
            }
            del r['choice'] 
            vote_records.append(r)
    df_votes = pd.DataFrame(vote_records)
    df_votes['proposal_id'] = df_votes.proposal.apply(lambda v: v['id']) 
    df_votes['vote_id'] = df_votes['id'] 
    # Different voters represent their choice amount in different ways. So we group by each unique combination of voter and proposal and normalize on a per vote level. 
    df_votes['proposal_voter_amount_total'] = df_votes.groupby(['proposal_id', 'voter'])['amount'].transform('sum')
    df_votes['choice_percent'] = df_votes.amount / df_votes.proposal_voter_amount_total * 100
    df_votes = df_votes.drop(columns=['proposal', 'id', 'vp_by_strategy', 'vp_state', 'proposal_voter_amount_total']) 
    df_votes = df_votes.rename(columns={'created': 'vote_created', 'vp': 'vote_power'})
    return df_votes

In [452]:
df_proposals = await get_snapshot_proposals()

AssertionError: 

In [ ]:
print(f"Number of votium snapshot proposals: {len(df_proposals)}")
df_proposals.head()

In [ ]:
df_choices = df_proposals.explode('proposal_choices').reset_index().rename(columns={'proposal_choices': 'choice'})
df_choices['choice_index'] = df_choices.groupby('proposal_id').cumcount()
df_choices = df_choices[['choice', 'choice_index', 'proposal_round', 'proposal_title', 'proposal_id', 'proposal_id_keccak256']]
df_choices['choice'] = df_choices.choice.str.lower()
df_choices.tail()

In [ ]:
gauge_data = requests.get('https://api.curve.fi/api/getAllGauges').json()
df_gauges = pd.DataFrame([
    {
        'gauge_name': d['name'].lower(), 
        'gauge_short_name': d['shortName'].lower(), 
        'gauge_address': d['swap_token']
    } 
    for _, d in data['data'].items() 
])
df_gauges.head()

In [ ]:
proposal_ids = [e.lower() for e in df_proposals.proposal_id.unique().tolist()]
df_votes = await get_snapshot_votes(proposal_ids)

In [ ]:
df_votes.head()

In [ ]:
df_snaps = df_votes.merge(df_proposals, how='left', on='proposal_id', validate='m:1')

def row_mapper(row): 
    row['vote_choice'] = row['proposal_choices'][int(row['choice_index'])]
    return row 

df_snaps = df_snaps.apply(row_mapper, axis=1)
df_snaps['vote_power_allocated'] = (df_snaps.choice_percent / 100) * df_snaps.vote_power 
df_snaps = df_snaps[[
    'proposal_round', 'proposal_title', 'proposal_start', 'proposal_end', 'voter', 'vote_power', 
    'vote_created', 'vote_choice', 'choice_percent', 'choice_index', 
    'vote_power_allocated', 
    'proposal_id', 'vote_id', 'proposal_id_keccak256'
]]
df_snaps = df_snaps.sort_values(['proposal_round', 'proposal_start']).reset_index(drop=True) 
# df_snaps.choice_index = df_snaps.choice_index.astype(int)

In [ ]:
(
    alt.Chart((
        df_snaps[['proposal_round', 'vote_choice', 'vote_power_allocated']]
        .groupby(['proposal_round', 'vote_choice']).sum()
        .reset_index() 
    ))
    .mark_bar()
    .encode(
        x="proposal_round:O", 
        y="vote_power_allocated:Q", 
        color="vote_choice:N", 
        tooltip=["vote_choice:N", "vote_power_allocated:Q"]
    )
    .properties(width=500) 
)

### Votium Bribes 

In [ ]:
df_snaps.head()

In [ ]:
# Get all votium voting epochs. Once we have validated that this set of epochs matches our set of snapshot proposals, 
# we need to merge this data with our snapshot proposal data. 
epoches = sg_votium.Query.epoches(first=1000, orderBy="initiatedAt", where={"bribeCount_gt": 0})
df_epoches = sg.query_df([epoches.id, epoches.initiatedAt], pagination_strategy=ShallowStrategy)
# df_epoches = remove_prefix(df_epoches, "epoches_")
df_epoches.epoches_initiatedAt = pd.to_datetime(df_epoches.epoches_initiatedAt, unit="s")
df_epoches['date'] = df_epoches.epoches_initiatedAt.dt.date
# Validate that the epoch dates for votium bribes match the proposal data we pulled from snapshot. `
epoch_dates = df_epoches.date.unique().tolist()
proposal_dates = df_snaps.proposal_start.unique().tolist()
d_exclude = pd.Timestamp('2021-11-08').date()
assert d_exclude in epoch_dates and not d_exclude in proposal_dates
epoch_dates.remove(d_exclude)
assert set(epoch_dates) == set(proposal_dates)
df_epoches = df_epoches.loc[df_epoches.date != d_exclude].reset_index(drop=True)
df_epoches = df_epoches.sort_values('date').reset_index().rename(columns={'index': 'round'})
df_epoches['round'] += 1
df_epoches.tail()

In [ ]:
# https://github.com/convex-community/convex-subgraph/blob/main/subgraphs/votium/src/mapping.ts

# Addresses associated with the frax protocol used for votium bribes 
# TODO: Frax controls some subset of the TVL in it's liquidity pools. Need to be cognizant of this because it leads 
#       to a rebate the lowers the cost of bribing. 
# TODO: Frax's vlCVX is not custodied in the investor custodian wallet. 
frax_bribe_addresses = [
    # ('comptroller', '0xb1748c79709f4ba2dd82834b8c82d4a505003f27'),
    # ('cvx locker amo', '0x7038c406e7e2c9f81571557190d26704bb39b8f3'),
    ('investor custodian', '0x5180db0237291A6449DdA9ed33aD90a38787621c'),
    ('frax1.eth', '0x234D953a9404Bf9DbC3b526271d440cD2870bCd2'),
]
# TODO: Check to see if there are bribes in FRAX 
bribes = sg_votium.Query.bribes(first=100000, where={
    "epoch_in": df_epoches.epoches_id.unique().tolist(), "token": ADDRESS_FXS
})
df_bribes = sg.query_df([bribes.id, bribes.amount, bribes.choiceIndex, bribes.epoch.id], pagination_strategy=ShallowStrategy)
df_bribes = remove_prefix(df_bribes, "bribes_")
df_bribes['tx_hash'] = df_bribes['id'].apply(lambda _id: _id.split('-')[0])
df_bribes.amount /= 1e18
df_bribes = df_bribes.drop(columns=['id'])
df_bribes = df_bribes.rename(columns={'amount': 'bribe_fxs', 'choiceIndex': 'choice_index'})
ddf(df_bribes.head())

In [ ]:
# Determine the "from" address for the transaction that contained the bribe 
tx_hashes = df_bribes.tx_hash.unique()

def get_tx(tx_hash): 
    return w3.eth.get_transaction(tx_hash)['from'] 

tx_from_map = {}
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    futures = {tx_hash: executor.submit(get_tx, tx_hash) for tx_hash in tx_hashes}
    for tx_hash, future in futures.items(): 
        tx_from_map[tx_hash] = future.result()

In [ ]:
df_bribes['from'] = df_bribes['tx_hash'].apply(lambda tx_hash: tx_from_map[tx_hash])
address_labels = {addr.lower(): name for name, addr in frax_bribe_addresses}
df_bribes['briber_label'] = df_bribes['from'].apply(lambda a: address_labels.get(a.lower()))
label_counts = df_bribes.briber_label.value_counts(dropna=False)
print("Label count for bribing address") 
print(label_counts)
print(f"Dropping {label_counts[None]} bribe(s)")
df_bribes = df_bribes.loc[~df_bribes.briber_label.isna()] 
df_bribes = df_bribes.drop(columns=['tx_hash', 'from']).sort_values(['briber_label', 'choice_index']).reset_index(drop=True)
df_bribes.head()

In [ ]:
# Match bribes with their corresponding round 
df_bribes_full = df_epoches.merge(df_bribes, how='right', left_on='epoches_id', right_on='epoch_id', validate='1:m') 
df_bribes_full.head()

In [ ]:
df_choices.head()

In [ ]:
df = (
    df_bribes_full.groupby(['round', 'choice_index'])['bribe_fxs'].sum()
    .reset_index()
    .merge(
        df_choices[['proposal_round', 'choice', 'choice_index']], 
        how='left',
        left_on=['round', 'choice_index'],
        right_on=['proposal_round', 'choice_index'], validate='m:1'
    )
    .merge(
        df_epoches[['round', 'epoches_initiatedAt']], 
        how='left', 
        on='round', 
        validate='m:1'
    )
    .drop(columns=['round'])
    .rename(columns={'epoches_initiatedAt': 'epoch_initiated_at'})
)
df['timestamp'] = pd.to_datetime(pd.to_datetime((df.epoch_initiated_at.astype(int) / 1e9).astype(int), unit='s').dt.date)
df.choice = df.choice.str.lower()
df.head()

In [ ]:
def process(df): 
    # Goal: For each choice in the snapshot proposal (i.e. a curve pool gauge), we want to pair the 
    #       choice name taken from the snapshot API with the address of the curve pool that choice 
    #       corresponds to. 
    
    # Remove irrelevant gauges  
    remove_choices = [
        "arbitrum-f-4pool", # 4pool never launched 
        "tusd", # Only bribed for in 1 round. Not really sure what to do about this one 
    ]
    df = df.loc[~df.choice.isin(remove_choices)]
    
    def preprocess_choice(choice):
        # Some older voting rounds used crvfrax while newer ones use fraxbp 
        choice = choice.replace('crvfrax', 'fraxbp')
        # Some older voting rounds prefixed factory pools with f- while newer rounds do not
        if choice.startswith("f-"): 
            choice = choice[2:]
        # Some older voting rounds showed addresses in form (0x...ab123)
        # whereas newer rounds use the form (0x...) without trailing values. 
        # Here, we remove trailing values if they exist 
        m = re.search('.*\\u2026([^\)]*)\)$', choice)
        if m: 
            choice = choice.replace(m.group(1), '') 
        return choice 
        
    df.choice = df.choice.apply(preprocess_choice)
    
    # We get the current set of gauges from the curve API. These names are what appear in the snapshot proposal 
    # This will get the most recent set of names but the structure of the names has changed over time. More work 
    # needs to be done to account for historical differences in naming pools 
    df = df.merge(df_gauges, how='left', left_on='choice', right_on='gauge_short_name')
    
    missed = df.loc[df.gauge_name.isna()].choice.unique()
 
    df_canonical_choices = pd.DataFrame(
        columns=['choice', 'canonical_choice'], 
        data=[
            ['frax', 'frax+usdc (0xdcef…)'], # fraxbp 
            ['d3pool', 'frax+fei+alusd (0xbaaa…)'], 
            ['fpifrax', 'frax+fpi (0xf861…)'], 
            ['2pool-frax', 'frax+usdc (0xdcef…)'], # fraxbp 
            ['fraxbpsusd', 'susd+fraxbp (0xe3c1…)'], 
            ['fraxbplusd', 'lusd+fraxbp (0x497c…)'], 
            ['fraxbpbusd', 'busd+fraxbp (0x8fdb…)'], 
            ['fraxbpape', 'apeusd+fraxbp (0x04b7…)'], 
            ['fraxbpalusd', 'alusd+fraxbp (0xb30d…)'], 
            ['fraxbpusdd', 'usdd+fraxbp (0x4606…)'], 
            ['fraxbptusd', 'tusd+fraxbp (0x33ba…)'], 
            ['fraxbpgusd', 'gusd+fraxbp (0x4e43…)'], 
        ]
    )    
    copy_cols = ['gauge_name', 'gauge_short_name', 'gauge_address']
    df = df.merge(df_canonical_choices, how='left', on='choice', validate='m:1')
    assert set(df.canonical_choice.dropna().unique()) == set(df_canonical_choices.canonical_choice.unique())
    df = pd.concat([
        # subset where we have already matched a choice to a gauge 
        df.loc[~df.gauge_name.isna()], 
        # subset where we have NOT matched a choice to a gauge 
        (
            df.loc[df.gauge_name.isna()]
            .drop(columns=df_gauges.columns.values.tolist()) 
            .merge(df_gauges, how='left', left_on='canonical_choice', right_on='gauge_short_name')
        )
    ])
    
    assert all(~df.gauge_address.isna()), "Unable to identify lp address for some choices"

    df = df.sort_values(['proposal_round', 'choice']).reset_index(drop=True)

    # miss.matrix(df)
    return df 

df = process(df) 

In [ ]:
# var = 'choice' 
var = 'gauge_short_name'
# var = 'gauge_address' 
(
    alt.Chart(df)
    .mark_bar()
    .encode(
        x="proposal_round:O", 
        y="bribe_fxs:Q", 
        color=f"{var}:N", 
        tooltip=[f'{var}:N', 'bribe_fxs:Q']
    )
)

In [ ]:
df = df[['proposal_round', 'timestamp', 'gauge_short_name', 'bribe_fxs', 'gauge_address']] 
df.head()

# miss.matrix(df)

In [ ]:
df_liquidity.dailyPoolSnapshots_timestamp = pd.to_datetime(df_liquidity.dailyPoolSnapshots_timestamp, unit='s')
df_liquidity = df_liquidity[
    ['dailyPoolSnapshots_timestamp', 'dailyPoolSnapshots_reservesUSD', 'pool_address', 'pools_name', 'pools_lpToken', 'pools_coins', 'pools_coinNames']
]
df_liquidity.head()

In [ ]:
# Map each bribe to liquidty data point for it's corresponding pool 
dff_join = pd.merge_asof(
    df.sort_values('timestamp').reset_index(drop=True),
    df_liquidity.sort_values('dailyPoolSnapshots_timestamp').reset_index(drop=True),
    left_by=['gauge_address'], 
    right_by=['pool_address'], 
    left_on=['timestamp'], 
    right_on=['dailyPoolSnapshots_timestamp', ], 
)[['proposal_round', 'dailyPoolSnapshots_timestamp', 'gauge_short_name', 'bribe_fxs', 'gauge_address']]
dff_join = dff_join.loc[~dff_join.dailyPoolSnapshots_timestamp.isna()]
print(len(dff_join))
dff_join.head()

In [ ]:
df_liquidity.head()

In [ ]:
dff_join.head()

In [ ]:
dff = (
    # 1. Ensure that the gauge information is attached to liquidity data 
    df_liquidity
    .merge(
        dff_join[['gauge_short_name', 'gauge_address']].drop_duplicates().reset_index(drop=True), 
        how='left', 
        left_on='pool_address', 
        right_on='gauge_address'
    ) 
    .merge(
        dff_join[['gauge_address', 'bribe_fxs', 'proposal_round', 'dailyPoolSnapshots_timestamp']], 
        how='left', 
        left_on=['pool_address', 'dailyPoolSnapshots_timestamp'], 
        right_on=['gauge_address', 'dailyPoolSnapshots_timestamp'], 
    ) 
    .drop(columns=['gauge_address_y'])
) 
dff = dff.loc[~dff.gauge_short_name.isna()]

# dff.bribe_fxs = dff.bribe_fxs.fillna(0)
# miss.matrix(dff) 
print(len(df_liquidity))
print(len(dff))

dff.head()

In [ ]:
# base = (
#     alt.Chart(dff)
#     .transform_calculate(stack_order="datum.pools_coinNames === 'crvFRAX' ? 0 : 1")
#     .encode(
        
#         color=alt.Color("pools_coinNames:N", scale=alt.Scale(scheme="tableau20")), 
#         tooltip=alt.Tooltip("dailyPoolSnapshots_reservesUSD:Q", format='$,d'), 
#         facet=alt.Facet('pools_symbol:N', columns=3),
#         order="stack_order:O", 
#     )
#     .resolve_scale(y="independent")
# )
# liquidity = (
#     base
#     .mark_area()
#     .encode(
#         x="dailyPoolSnapshots_timestamp:T", 
#         y="dailyPoolSnapshots_reservesUSD:Q"
#     )
#     # .resolve_scale(y="independent")
# )
# bribes = (
#     base
#     .mark_bar()
#     .encode(
#         x="dailyPoolSnapshots_timestamp:T", 
#         y="bribe_fxs:Q",
#     )
#     .resolve_scale(y="independent")
# )

# # miss.matrix(dff)

# liquidity | bribes 

In [ ]:
miss.matrix(dff)

In [ ]:
dff.head()

In [ ]:
max_bribe

In [ ]:
ncols = 3 
rows = []
row = []
gnames = dff.gauge_short_name.unique().tolist()

max_tvl = (
    dff.groupby(['dailyPoolSnapshots_timestamp', 'gauge_short_name'])['dailyPoolSnapshots_reservesUSD'].sum().max()
)
max_bribe = (
    dff['bribe_fxs'].max() 
)

while gnames: 
    gname = gnames.pop()
    # add data for this gauge to current row 
    sdf = dff.loc[dff.gauge_short_name == gname] 
    base = (
        alt.Chart(sdf)
        .transform_calculate(stack_order="datum.pools_coinNames === 'crvFRAX' ? 0 : 1")
        .encode(
            x="dailyPoolSnapshots_timestamp:T", 
            color=alt.Color("pools_coinNames:N", scale=alt.Scale(scheme="tableau20")), 
            order="stack_order:O", 
        )
        .properties(width=250, height=150) 
    )
    liquidity = base.mark_area().encode(
        y=alt.Y("dailyPoolSnapshots_reservesUSD:Q", scale=alt.Scale(domain=[0, max_tvl * 1.05])),
        tooltip=alt.Tooltip("dailyPoolSnapshots_reservesUSD:Q", format='$,d'), 
    )
    bribes = (
        base
        .transform_filter("datum.pools_coinNames === 'crvFRAX'")
        .mark_bar()
        .encode(
            y=alt.Y("bribe_fxs:Q", scale=alt.Scale(domain=[0, max_bribe * 1.05])),
            tooltip=alt.Tooltip("bribe_fxs:Q", format=',d'), 
        )
    )
    row.append(alt.vconcat(liquidity, bribes).resolve_scale(x="shared").resolve_axis(x='shared'))
    if len(row) == ncols: 
        rows.append(row) 
        row = []
    
rows = [alt.hconcat(*row) for row in rows]
chart = alt.vconcat(*rows)    
chart


In [ ]:
sdf[~sdf.bribe_fxs.isna()].head()

# miss.matrix(sdf)
# ddf(df_liquidity.loc[df_liquidity.pools_coinNames == 'ApeUSD'].head(20))